#Lecture 7- Spring 2023
#Villas-Boas

#Lecture 7 EEO 118 Spring 2023

#if in your computer change into your working directory
#change directory
#-------------------------------------------
#set your working directory
#-------------------------------------------
#setwd("/Users/sberto/Desktop/")
#setwd("/Users/sofiavillas-boas/Dropbox/EEP118_Spring2023/Lectures/Lecture7")

In [15]:
# Load the 'pacman' package
library(pacman) 
#packages to use load them now using the pacman "manager"
p_load(dplyr, readr) 
#Another great feature of p_load(): if you try to load a package that is not installed on your machine, p_load() install the package for you, rather than throwing an error. For instance, let’s install and load one final package named ggplot2. 
p_load(ggplot2)



#set scientific display off, thank you Roy
options(scipen=999)


pacman::p_load(lfe, lmtest, haven, sandwich, tidyverse)
# lfe for running fixed effects regression
# lmtest for displaying robust SE in output table
# haven for loading in dta files
# sandwich for producing robust Var-Cov matrix
# tidyverse for manipulating data and producing plots

ERROR: Error in library(pacman): there is no package called ‘pacman’


In [16]:
library(haven)
library(ggplot2)

In [6]:
#-------------------------------------------
#1. Read in data
#-------------------------------------------
my_data <- read_dta("dataLecture7.dta")
head(my_data)

state,county,fips,trump16,clinton16,otherpres16,romney12,obama12,otherpres12,demsen16,⋯,age65andolder_pct,median_hh_inc,clf_unemploy_pct,lesshs_pct,lesscollege_pct,lesshs_whites_pct,lesscollege_whites_pct,rural_pct,ruralurban_cc,bh_pct
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Alabama,Autauga,1001,18172,5936,865,17379,6363,190,6331,⋯,13.97845556,53099,5.591657,12.41704602,75.40722865,10.00211164,74.06560146,42.00216232,2,20.94316
Alabama,Baldwin,1003,72883,18458,3874,66016,18424,898,19145,⋯,18.71485139,51365,6.286843,9.972417933,70.45288897,7.842226994,68.40560728,42.27909911,3,13.59230
Alabama,Barbour,1005,5454,4871,144,5550,5912,47,4777,⋯,16.52889457,33956,12.824738,26.23592756,87.13221298,19.57975167,81.36474583,67.7896347,6,52.19809
Alabama,Bibb,1007,6738,1874,207,6132,2202,86,2082,⋯,14.8856991,39776,7.146827,19.3015873,88,15.02049009,87.47177386,68.35260746,1,23.43612
Alabama,Blount,1009,22859,2156,573,20757,2970,279,2980,⋯,17.19291557,46212,5.953833,19.96858533,86.95024321,16.64336837,86.16360993,89.95150204,1,10.28525
Alabama,Bullock,1011,1140,3530,40,1251,4061,10,3364,⋯,15.13457165,29335,13.258520,33.43788354,89.74498841,11.7803838,79.15778252,51.37438153,6,75.62547


In [ ]:
#generate variables Y and x1
my_data$Y<-my_data$trump16/my_data$clinton16
my_data$x1<-my_data$romney12/my_data$obama12

In [ ]:
slide12<-plot(my_data$x1,my_data$Y) 

In [ ]:
#use non missing values only from now on
my_data2<-my_data[complete.cases(my_data),]
my_data3 <- my_data2[my_data2$Y !=Inf,] 
#my_data3 <-my_data2[my_data2$white_pct!=NA]

In [ ]:
summary(my_data3)
#regression full model
regfull<-lm(Y ~ x1 + white_pct, my_data3)
summary(regfull)

In [ ]:
#regression no percent white as control
regsmall<-lm(Y ~ x1, my_data3)
summary(regsmall)

In [ ]:
#collinearity Slides

#Baseline Model 
regBase <- lm(Y~x1+female_pct+white_pct, my_data3)
summary(regBase)

In [ ]:
#Alternative Model – Perfect collinearity
my_data3$male_pct=1-my_data3$female_pct
regPC <- lm(Y~x1+female_pct+male_pct+white_pct, my_data3)
summary(regPC)

In [ ]:
#Alternative Model – Multi collinearity
regMC <- lm(Y~x1+female_pct+white_pct+bh_pct, my_data3)
#where corr(white_pct,bh_pct)=-0.92
summary(regMC)

In [ ]:
#for graph in slide 28
#get the predicted Y hats
my_data3$Yhat<-regBase$fitted.values

#put Y and Y hat on same graph and x1 on horizontal axis
#make combined scatter plot of Y data and fitted values of Y (Yhat)
#given regression estimates usinf X1, white_pct and female_pct
scatter_data_fittedVals <- ggplot(data = my_data3) + geom_point(aes(x=x1, y=Y, color = "data")) +
  geom_point(aes(x=x1, y=Yhat, color = "fitted")) + 
  xlab("x1=Ratio Rodney to Obama Votes in 2012") + ylab("Y=Ratio Trump to Clinton Votes in 2016 ") +
  ggtitle("Y (Red) and Predicted Y (Blue) and x1")
scatter_data_fittedVals